# Trying to remove from multiple layers

In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
y_train, y_test = tf.one_hot(y_train, 10), tf.one_hot(y_test, 10)

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [5]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
    
tester_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [6]:
ow = np.array(model.get_weights())
print(ow.shape)

(6,)


In [7]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4953 - accuracy: 0.8233 - auc: 0.9850
Epoch 2/5
1103/1875 [================>.............] - ETA: 2s - loss: 0.3729 - accuracy: 0.8637 - auc: 0.9908

KeyboardInterrupt: 

In [24]:
l, a, auc = model.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights = model.get_weights()
weight_len = len(or_weights) - 3
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
amounts = []
places = []
layer_sizes = [64, 128]
for layer, size in enumerate(layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights)
        w[weight_len - (2*layer+1)][:,i] = 0
        w[weight_len - 2*layer][i] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_test, y_test, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))

40/40 - 0s - loss: 0.3920 - accuracy: 0.8633 - auc_2: 0.9891
Node 0: -0.004817008972167969
Node 1: -0.0034227371215820312
Node 2: -9.521842002868652e-05
Node 3: -0.0028116703033447266
Node 4: -0.002624303102493286
Node 5: 0.0
Node 6: -7.203221321105957e-05
Node 7: -0.003041297197341919
Node 8: 0.003934323787689209
Node 9: -0.02047094702720642
Node 10: 9.429454803466797e-05
Node 11: -0.004074901342391968
Node 12: -0.014137089252471924
Node 13: -0.017557114362716675
Node 14: -0.00905466079711914
Node 15: 2.327561378479004e-05
Node 16: 0.011436671018600464
Node 17: 0.0008215010166168213
Node 18: 0.002939879894256592
Node 19: -0.009530246257781982
Node 20: -0.00716128945350647
Node 21: -0.009292006492614746
Node 22: -0.009544789791107178
Node 23: -0.0009729862213134766
Node 24: 0.0014873147010803223
Node 25: -0.006831735372543335
Node 26: -0.0012098848819732666
Node 27: -0.003622770309448242
Node 28: -0.006294578313827515
Node 29: -0.003550410270690918
Node 30: -0.001171410083770752
Node 3

KeyboardInterrupt: 

In [19]:
for i, (nz, nw, ni) in enumerate(reversed(amounts)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 12
Worse Nodes: 40
Important Nodes: 76
######### LAYER 1 #########
Zero Nodes: 3
Worse Nodes: 20
Important Nodes: 41


In [18]:
for i, e in enumerate(reversed(amounts)):
    print(i, e)

0 (12, 40, 76)
1 (3, 20, 41)


In [30]:
loss, acc, auc = model.evaluate(x_train, y_train, verbose=2, batch_size=512)
original2 = model.get_weights()
tol = -1e-5
bas2 = [acc]
bls2 = [loss]
best_weights2 = model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
for layer, size in enumerate(layer_sizes):
    end_not_reached = True
    improved = False
    current_pos = 0
    best_pos = -1
    best_change = tol
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(layer_sizes) - layer}')
    while end_not_reached or improved:
        if not(end_not_reached):
            end_not_reached = True
            improved = False
            current_pos = 0
            layer_sizes[layer] -= 1
            size -= 1
            nodes_removed2 += [best_pos]
            best_weights2[weight_len - (2*layer+1)][:,best_pos] = 0
            best_weights2[weight_len - 2*layer][best_pos] = 0
            best_pos = -1
            #tol -= best_change
            ol = best_loss
            oa = best_acc
            bas2 += [best_acc]
            bls2 += [best_loss]
            print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
            best_change = tol
            num_removed2 += 1
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights2)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_train, y_train, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.3*(na - oa) + 0.7*(ol - nl) > best_change:
            best_change = 0.3*(na - oa) + 0.7*(ol - nl)
            print(best_change)
            best_pos = current_pos
            improved = True
            best_acc = na
            best_loss = nl
            print("Found something better")
        current_pos += 1
        if current_pos - num_removed2 >= size:
            end_not_reached = False
        if current_pos%200 == 0:
            print("Did 200 iterations")
    amounts.append(num_removed2)
    places.append(nodes_removed2)

tester_model.set_weights(best_weights2)
loss2, acc2, auc2 = tester_model.evaluate(x_test, y_test, verbose=2)

118/118 - 1s - loss: 0.3098 - accuracy: 0.8856 - auc_2: 0.9934
Considering layer 2
2.0861625671386717e-08
Found something better
0.0023130923509597774
Found something better
0.007125061750411986
Found something better
0.016171675920486447
Found something better
Improvement has occured!! Accuracy: 0.8918833136558533 --- Loss: 0.289421409368515 --- Change: 0.016171675920486447 --- New tol: -1e-05
0.0
Found something better
0.0019133180379867552
Found something better
0.0040796220302581785
Found something better
0.006046676635742187
Found something better
Improvement has occured!! Accuracy: 0.8943166732788086 --- Loss: 0.2818261682987213 --- Change: 0.006046676635742187 --- New tol: -1e-05
0.0
Found something better
0.0012735426425933838
Found something better
0.003202742338180542
Found something better
Improvement has occured!! Accuracy: 0.8964999914169312 --- Loss: 0.27818652987480164 --- Change: 0.003202742338180542 --- New tol: -1e-05
0.0
Found something better
0.0009826242923736573
F

In [31]:
y_pred = model.predict(x_test)

In [18]:
y_test_flat = np.argmax(y_test, axis=1)

In [40]:
print(y_test_flat)

tf.Tensor([9 2 1 ... 8 1 5], shape=(10000,), dtype=int64)


In [38]:
print(y_pred.shape)

(10000, 10)


In [43]:
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.803, 0.986, 0.791, 0.745, 0.864, 0.94, 0.614, 0.951, 0.973, 0.966]


In [45]:
y_pred = tester_model.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.803, 0.968, 0.788, 0.896, 0.805, 0.949, 0.679, 0.957, 0.969, 0.957]


In [49]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model2.set_weights(best_weights2)

In [79]:
amounts

[11, 23]

In [54]:
l3, a3, auc3 = model2.evaluate(x_train, y_train, verbose=2, batch_size=256)
or_weights2 = model2.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
amounts2 = []
places2 = []
layer_sizes = [64, 128]
for layer, size in enumerate(layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights2)
        w[weight_len - (2*layer+1)][:,i] = 0
        w[weight_len - 2*layer][i] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_train, y_train, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a3) + 1.*(l3 - nl))
        change = l3 - nl
        if not i in places[layer]:
            if change <= tol_high and change >= tol_low:
                num_zeros += 1
                z += [i]
            elif change > 0:
                num_worse += 1
                wr += [i]
            else:
                num_important += 1
                imp += [i]
    amounts2.append((num_zeros, num_worse, num_important))
    places2.append((z, wr, imp))

235/235 - 1s - loss: 0.2722 - accuracy: 0.9018 - auc_5: 0.9955
Node 0: -0.0007068216800689697
Node 1: -0.00951436161994934
Node 2: -0.00010100007057189941
Node 3: -0.0030525028705596924
Node 4: -0.004624217748641968
Node 5: 0.0
Node 6: -0.0004836618900299072
Node 7: -0.000919640064239502
Node 8: 0.0
Node 9: -0.014550328254699707
Node 10: 0.0
Node 11: -0.0030814409255981445
Node 12: -0.006567031145095825
Node 13: -0.004613637924194336
Node 14: -0.0046923160552978516
Node 15: -0.007797300815582275
Node 16: 0.0
Node 17: -0.003049790859222412
Node 18: -0.008292704820632935
Node 19: -0.0058487653732299805
Node 20: -0.008133083581924438
Node 21: -0.002508878707885742
Node 22: -0.002284616231918335
Node 23: -0.0005620419979095459
Node 24: -0.0030630826950073242
Node 25: -0.008069068193435669
Node 26: -0.0029172301292419434
Node 27: -0.002832561731338501
Node 28: -0.004408299922943115
Node 29: -0.0020445585250854492
Node 30: -0.0010837316513061523
Node 31: 0.0
Node 32: -0.0050369203090667725
N

In [55]:
for i, (nz, nw, ni) in enumerate(reversed(amounts2)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 1
Worse Nodes: 0
Important Nodes: 104
######### LAYER 1 #########
Zero Nodes: 0
Worse Nodes: 0
Important Nodes: 53


In [58]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.set_weights(original2)

## Greedy removal

In [75]:
loss, acc, auc = model.evaluate(x_train, y_train, verbose=2, batch_size=512)
original2 = model.get_weights()
tol = -1e-5
layer_sizes = [64, 128]
bas2 = [acc]
bls2 = [loss]
best_weights4 = model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts3 = []
places3 = []
for layer, size in enumerate(layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights4)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_train, y_train, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.3*(na - oa) + 0.7*(ol - nl) >= tol:
            best_change = 0.3*(na - oa) + 0.7*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights4[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights4[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts3.append(num_removed2)
    places3.append(nodes_removed2)

tester_model.set_weights(best_weights4)
loss2, acc2, auc2 = tester_model.evaluate(x_test, y_test, verbose=2)

118/118 - 0s - loss: 0.3098 - accuracy: 0.8856 - auc_7: 0.9934
Considering layer 2
Improvement has occured!! Accuracy: 0.8855500221252441 --- Loss: 0.30980950593948364 --- Change: 2.0861625671386717e-08 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8860999941825867 --- Loss: 0.30674082040786743 --- Change: 0.002313071489334106 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8886666893959045 --- Loss: 0.2979861795902252 --- Change: 0.00689825713634491 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8897500038146973 --- Loss: 0.2968910336494446 --- Change: 0.001091596484184265 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8911666870117188 --- Loss: 0.293424516916275 --- Change: 0.002851566672325134 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8911666870117188 --- Loss: 0.293424516916275 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8916833400726318 --- Loss: 0.2924406826496124 --- Change: 0.0008436799049

In [76]:
model3 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model3.set_weights(best_weights4)

In [82]:
places3

[[5, 8, 16, 18, 31, 36, 43, 10, 47, 50, 56, 60],
 [1,
  3,
  6,
  12,
  13,
  15,
  24,
  26,
  31,
  35,
  36,
  39,
  43,
  48,
  49,
  58,
  71,
  75,
  79,
  81,
  83,
  91,
  115,
  126]]

In [80]:
l4, a4, auc4 = model3.evaluate(x_train, y_train, verbose=2, batch_size=256)
or_weights3 = model3.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
amounts4 = []
places4 = []
layer_sizes = [64, 128]
for layer, size in enumerate(layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights3)
        w[weight_len - (2*layer+1)][:,i] = 0
        w[weight_len - 2*layer][i] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_train, y_train, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.3*(na - a4) + 0.7*(l4 - nl))
        change = l4 - nl
        if not i in places[layer]:
            if change <= tol_high and change >= tol_low:
                num_zeros += 1
                z += [i]
            elif change > 0:
                num_worse += 1
                wr += [i]
            else:
                num_important += 1
                imp += [i]
    amounts4.append((num_zeros, num_worse, num_important))
    places4.append((z, wr, imp))

235/235 - 1s - loss: 0.2792 - accuracy: 0.8990 - auc_9: 0.9952
Node 0: -0.00027801692485809323
Node 1: -0.010227012634277343
Node 2: -9.000003337860108e-05
Node 3: -0.0030906170606613157
Node 4: -0.003475725650787353
Node 5: 0.0
Node 6: -0.0002203613519668579
Node 7: -0.0007068723440170287
Node 8: 0.0
Node 9: -0.009521567821502683
Node 10: 0.0
Node 11: -0.001864698529243469
Node 12: -0.0046282678842544545
Node 13: -0.002602782845497131
Node 14: -0.006760761141777038
Node 15: -0.004790139198303222
Node 16: 0.0
Node 17: -0.0018842697143554687
Node 18: 0.0
Node 19: -0.00422491431236267
Node 20: -0.005683809518814087
Node 21: -0.0019903302192687987
Node 22: -0.0012815326452255249
Node 23: -0.00035728812217712403
Node 24: -0.0016927331686019897
Node 25: -0.005865535140037537
Node 26: -0.002036064863204956
Node 27: -0.002338472008705139
Node 28: -0.005013236403465271
Node 29: -0.0013986796140670777
Node 30: -0.0008497655391693115
Node 31: 0.0
Node 32: -0.008186012506484985
Node 33: -0.014611

In [81]:
for i, (nz, nw, ni) in enumerate(reversed(amounts4)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 3
Worse Nodes: 0
Important Nodes: 102
######### LAYER 1 #########
Zero Nodes: 1
Worse Nodes: 0
Important Nodes: 52


In [83]:
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.803, 0.968, 0.788, 0.896, 0.805, 0.949, 0.679, 0.957, 0.969, 0.957]


In [84]:
y_pred = tester_model.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.804, 0.959, 0.769, 0.882, 0.797, 0.955, 0.698, 0.953, 0.972, 0.959]


# 3 layer

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [9]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
    
tester_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5026 - accuracy: 0.8207 - auc_2: 0.9846
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3689 - accuracy: 0.8644 - auc_2: 0.9911
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3334 - accuracy: 0.8767 - auc_2: 0.9925
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3151 - accuracy: 0.8819 - auc_2: 0.9934
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2954 - accuracy: 0.8900 - auc_2: 0.9941


In [11]:
l, a, auc = model.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights = model.get_weights()
weight_len = len(or_weights) - 3
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
amounts = []
places = []
layer_sizes = [32, 64, 128]
for layer, size in enumerate(layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights)
        w[weight_len - (2*layer+1)][:,i] = 0
        w[weight_len - 2*layer][i] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_test, y_test, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))

40/40 - 0s - loss: 0.3587 - accuracy: 0.8697 - auc_2: 0.9905
Node 0: 0.005494028329849243
Node 1: -0.0005543828010559082
Node 2: -0.02857455611228943
Node 3: -0.011651873588562012
Node 4: -0.02465301752090454
Node 5: -0.01774802803993225
Node 6: -0.013079166412353516
Node 7: 0.0017140507698059082
Node 8: -2.9802322387695312e-08
Node 9: -0.0050334632396698
Node 10: 0.00021898746490478516
Node 11: -0.008084803819656372
Node 12: -0.03289198875427246
Node 13: -0.03555145859718323
Node 14: -0.005437731742858887
Node 15: -0.02062740921974182
Node 16: -0.00033780932426452637
Node 17: -0.014653593301773071
Node 18: -0.006222337484359741
Node 19: -0.008284389972686768
Node 20: -0.010763972997665405
Node 21: 0.0006979405879974365
Node 22: 0.00496634840965271
Node 23: -0.018248677253723145
Node 24: -0.027724236249923706
Node 25: -0.0037917792797088623
Node 26: -0.0014519691467285156
Node 27: -0.007471948862075806
Node 28: 0.0005661845207214355
Node 29: -0.0015793442726135254
Node 30: -0.002554237

In [12]:
for i, (nz, nw, ni) in enumerate(reversed(amounts)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 11
Worse Nodes: 33
Important Nodes: 84
######### LAYER 1 #########
Zero Nodes: 2
Worse Nodes: 19
Important Nodes: 43
######### LAYER 2 #########
Zero Nodes: 1
Worse Nodes: 6
Important Nodes: 25


In [14]:
loss, acc, auc = model.evaluate(x_train, y_train, verbose=2, batch_size=512)
original2 = model.get_weights()
tol = -1e-5
layer_sizes = [32, 64, 128]
bas2 = [acc]
bls2 = [loss]
best_weights4 = model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts3 = []
places3 = []
for layer, size in enumerate(layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights4)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_train, y_train, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.3*(na - oa) + 0.7*(ol - nl) >= tol:
            best_change = 0.3*(na - oa) + 0.7*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights4[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights4[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts3.append(num_removed2)
    places3.append(nodes_removed2)

tester_model.set_weights(best_weights4)
loss2, acc2, auc2 = tester_model.evaluate(x_test, y_test, verbose=2)

118/118 - 0s - loss: 0.2792 - accuracy: 0.8946 - auc_2: 0.9946
Considering layer 3
Improvement has occured!! Accuracy: 0.8958666920661926 --- Loss: 0.27641811966896057 --- Change: 0.0023313969373703 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8958666920661926 --- Loss: 0.27642154693603516 --- Change: -2.3990869522094727e-06 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.895883321762085 --- Loss: 0.2762441337108612 --- Change: 0.0001291781663894653 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8981833457946777 --- Loss: 0.2741703391075134 --- Change: 0.002141663432121277 --- New tol: -1e-05
Layer optimized
Considering layer 2
Improvement has occured!! Accuracy: 0.8981999754905701 --- Loss: 0.27417001128196716 --- Change: 5.218386650085449e-06 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.8994666934013367 --- Loss: 0.2740398347377777 --- Change: 0.00047113895416259763 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.90031665563583

In [15]:
tester_model.evaluate(x_train, y_train, verbose=2)

1875/1875 - 4s - loss: 0.2642 - accuracy: 0.9041 - auc_3: 0.9958


[0.2641812562942505, 0.9040666818618774, 0.995792806148529]

In [19]:
y_pred = model.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.73, 0.965, 0.776, 0.907, 0.811, 0.888, 0.732, 0.966, 0.967, 0.955]


In [20]:
y_pred = tester_model.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.797, 0.968, 0.777, 0.892, 0.84, 0.953, 0.679, 0.956, 0.965, 0.954]


# CIFAR 10 data - 3 layer

In [23]:
cifar = tf.keras.datasets.cifar10
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
y_train, y_test = tf.one_hot(y_train.flatten(), 10), tf.one_hot(y_test.flatten(), 10)
y_test_flat = np.argmax(y_test, axis=1)

In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [15]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
    
tester_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [16]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
1563/1563 [==============================] - 4s 3ms/step - loss: 2.0611 - accuracy: 0.2287 - auc_4: 0.7066
Epoch 2/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7864 - accuracy: 0.3545 - auc_4: 0.8051
Epoch 3/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7018 - accuracy: 0.3863 - auc_4: 0.8264
Epoch 4/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.6400 - accuracy: 0.4099 - auc_4: 0.8403
Epoch 5/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5874 - accuracy: 0.4295 - auc_4: 0.8513
Epoch 6/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.5505 - accuracy: 0.4434 - auc_4: 0.8590
Epoch 7/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5182 - accuracy: 0.4566 - auc_4: 0.8653
Epoch 8/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4895 - accuracy: 0.4649 - auc_4: 0.8708
Epoch 9/20
1563/1563 [==============================] - 

In [18]:
l, a, auc = model.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights = model.get_weights()
weight_len = len(or_weights) - 3
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
amounts = []
places = []
layer_sizes = [16, 32, 64, 128, 256]
for layer, size in enumerate(layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights)
        w[weight_len - (2*layer+1)][:,i] = 0
        w[weight_len - 2*layer][i] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_test, y_test, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))

40/40 - 0s - loss: 1.4547 - accuracy: 0.4852 - auc_4: 0.8793
Node 0: 0.001023411750793457
Node 1: -5.8650970458984375e-05
Node 2: 2.658367156982422e-05
Node 3: -0.37429845333099365
Node 4: -0.4857388734817505
Node 5: -0.3562051057815552
Node 6: -0.02086508274078369
Node 7: 0.00011420249938964844
Node 8: -0.5088027715682983
Node 9: 0.0
Node 10: -0.29511559009552
Node 11: 0.0007669925689697266
Node 12: -0.062075018882751465
Node 13: -0.018933534622192383
Node 14: 0.0
Node 15: -0.1759035587310791
Node 0: -0.14026224613189697
Node 1: 9.167194366455078e-05
Node 2: -0.09598207473754883
Node 3: -0.02030313014984131
Node 4: -0.00032591819763183594
Node 5: -0.019550204277038574
Node 6: 0.0010339021682739258
Node 7: -0.003211498260498047
Node 8: -3.790855407714844e-05
Node 9: 5.960464477539062e-07
Node 10: -0.05502617359161377
Node 11: -0.47560954093933105
Node 12: 0.0
Node 13: -0.061805009841918945
Node 14: 0.00011658668518066406
Node 15: -0.023238182067871094
Node 16: 0.0
Node 17: 0.0002957582

In [19]:
for i, (nz, nw, ni) in enumerate(reversed(amounts)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 213
Worse Nodes: 2
Important Nodes: 41
######### LAYER 1 #########
Zero Nodes: 31
Worse Nodes: 21
Important Nodes: 76
######### LAYER 2 #########
Zero Nodes: 4
Worse Nodes: 17
Important Nodes: 43
######### LAYER 3 #########
Zero Nodes: 6
Worse Nodes: 6
Important Nodes: 20
######### LAYER 4 #########
Zero Nodes: 2
Worse Nodes: 4
Important Nodes: 10


In [20]:
loss, acc, auc = model.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model.get_weights()
tol = -1e-5
layer_sizes = [16, 32, 64, 128, 256]
bas2 = [acc]
bls2 = [loss]
best_weights4 = model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts3 = []
places3 = []
for layer, size in enumerate(layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights4)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.3*(na - oa) + 0.7*(ol - nl) >= tol:
            best_change = 0.3*(na - oa) + 0.7*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights4[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights4[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts3.append(num_removed2)
    places3.append(nodes_removed2)

tester_model.set_weights(best_weights4)
loss2, acc2, auc2 = tester_model.evaluate(x_test, y_test, verbose=2)

20/20 - 0s - loss: 1.4547 - accuracy: 0.4852 - auc_4: 0.8793
Considering layer 5
Improvement has occured!! Accuracy: 0.48500001430511475 --- Loss: 1.4536778926849365 --- Change: 0.0006563127040863036 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.48500001430511475 --- Loss: 1.4536432027816772 --- Change: 2.428293228149414e-05 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.48500001430511475 --- Loss: 1.4535399675369263 --- Change: 7.226467132568359e-05 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.48500001430511475 --- Loss: 1.4535399675369263 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.48489999771118164 --- Loss: 1.4529216289520264 --- Change: 0.00040283203125 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.48489999771118164 --- Loss: 1.4529216289520264 --- Change: 0.0 --- New tol: -1e-05
Layer optimized
Considering layer 4
Improvement has occured!! Accuracy: 0.48510000109672546 --- Loss: 1.4528322219848633 --- C

In [21]:
tester_model.evaluate(x_train, y_train, verbose=2)

1563/1563 - 4s - loss: 1.3094 - accuracy: 0.5359 - auc_5: 0.9035


[1.3094489574432373, 0.5358800292015076, 0.9034848809242249]

In [24]:
y_pred = model.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.474, 0.509, 0.252, 0.243, 0.403, 0.347, 0.745, 0.545, 0.745, 0.589]


In [25]:
y_pred = tester_model.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.562, 0.61, 0.256, 0.308, 0.525, 0.36, 0.611, 0.528, 0.584, 0.586]


In [27]:
amounts3

[6, 13, 15, 45, 214]

In [28]:
layer_sizes

[10, 19, 49, 83, 42]

In [33]:
for i, size in enumerate(reversed(layer_sizes)):
    print(f"Layer {i}: Modified: {size} nodes")
    print(f"         Original: {size + amounts3[4-i]} nodes")

Layer 0: Modified: 42 nodes
         Original: 256 nodes
Layer 1: Modified: 83 nodes
         Original: 128 nodes
Layer 2: Modified: 49 nodes
         Original: 64 nodes
Layer 3: Modified: 19 nodes
         Original: 32 nodes
Layer 4: Modified: 10 nodes
         Original: 16 nodes


In [45]:
old_param = 32*32*3*256+256*128+128*64+64*32+32*16+16*10 + 256+128+64+32+16+10
new_param = 32*32*3*42+42*83+83*49+49*19+19*10+10*10 + 42+83+49+19+10+10

print(f"Old num parameters: {old_param}")
print(f"New num parameters: {new_param}")
print(f"Percentage reduction: {round((1 - new_param/old_param) * 100)}%")

Old num parameters: 830618
New num parameters: 138011
Percentage reduction: 83%


In [35]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 256)               786688    
_________________________________________________________________
dense_21 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_22 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_23 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_24 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_25 (Dense)             (None, 10)               